In [13]:
#讀取資料
import pandas as pd
import pandas.io.sql as pd_sql
import sqlite3 as sql

#從CSV讀取
df = pd.read_csv('fp_demo3.csv')

#從sqLite讀取
#conn = sql.connect("twse.db")
#df = pd.read_sql_query("select * from demo3;", conn)

print(df.head())

         日期        成交股數           成交金額    開盤價    最高價    最低價    收盤價  漲跌價差  \
0  2017/9/1  17,805,405  3,851,834,126  215.0  217.5  215.0  216.5   0.0   
1  2017/9/4  15,053,185  3,273,164,145  216.5  218.0  215.5  217.5   1.0   
2  2017/9/5  34,514,275  7,503,488,603  218.0  218.0  216.5  218.0   0.5   
3  2017/9/6  28,148,528  6,117,444,609  216.5  218.0  216.0  217.0  -1.0   
4  2017/9/7  19,198,540  4,170,321,680  218.0  218.5  216.0  217.0   0.0   

    成交筆數  美元／新台幣   人民幣／新台幣    歐元／美元    美元／日幣    英鎊／美元    澳幣／美元    美元／港幣  \
0  6,956  30.156  4.584315  1.18920  110.100  1.29145  0.79360  7.82595   
1  8,635  30.101  4.608885  1.18960  109.460  1.29420  0.79560  7.82535   
2  7,065  30.070  4.586457  1.18810  109.455  1.29255  0.79635  7.82525   
3  6,999  30.105  4.606336  1.19440  108.710  1.30420  0.79910  7.82460   
4  6,586  30.062  4.619808  1.19485  109.015  1.30460  0.80015  7.81960   

    美元／人民幣    美元／南非幣    紐幣／美元  
0  6.57810  12.94610  0.71595  
1  6.53110  12.96600  0.7170

In [61]:
#定義準備train data方法
def regData(x):
    #格式化數字
    for i, row in x.iterrows():
        x.loc[i, '成交股數'] = row['成交股數'].replace(',', '')
        x.loc[i, '成交筆數'] = row['成交筆數'].replace(',', '')
    #做normalization(第i天資料/i-1天資料)
    for i, row in X.iterrows():
        if i <= len(X)-2:
            x.loc[i, '成交股數'] = int(x.loc[i+1, '成交股數'])/int(x.loc[i, '成交股數'])
            x.loc[i, '成交筆數'] = int(x.loc[i+1, '成交筆數'])/int(x.loc[i, '成交筆數'])
            x.loc[i, '美元／新台幣'] = x.loc[i+1, '美元／新台幣']/x.loc[i, '美元／新台幣']
    return x.head(30) #不取最後一筆

#定義LinearRegression對應data
def linReg(y):
    y.drop(0, axis=0, inplace=True)
    y = y.reset_index(drop=True)
    return y

#定義LogisticRegression對應data
def logReg(y):
    #將價差=0定義為 1 (漲), 價差<=0定義為 0 (跌)
    y.drop(0, axis=0, inplace=True)
    y = y.reset_index(drop=True)
    for i, row in y.iterrows():
        if y.loc[i, '漲跌價差'] == 0:
            y.loc[i, '漲跌價差'] = 1
        else:
            y.loc[i, '漲跌價差'] = 0
    return y['漲跌價差']
    
#取得必要欄位，以前30筆作為training data
X = df[['成交股數', '成交筆數', '美元／新台幣']].head(31)
X = regData(X)

#y = linReg(df[['收盤價']].head(31)) #LinearRegression
y = logReg(df[['漲跌價差']].head(31)) #LogisticRegression

#分割資料
from sklearn.model_selection import train_test_split

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

In [53]:
#建立LinearRegression模型
from sklearn.linear_model import LinearRegression, LogisticRegression

#regression_model = LinearRegression() #LinearRegression
regression_model = LogisticRegression() #LogisticRegression
regression_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [54]:
#各data係數
for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

The coefficient for 成交股數 is 0.010982436614916647
The coefficient for 成交筆數 is -0.7053547042303494
The coefficient for 美元／新台幣 is -0.3072815705648836


In [55]:
#截距
intercept = regression_model.intercept_[0]

print("The intercept for our model is {}".format(intercept))

The intercept for our model is -0.303918361823181


In [56]:
#用training data算準確率
predictions = regression_model.predict(X_train)
regression_model.score(X_train, y_train)
#資料零散程度大且資料量不夠可能造成準確率較低

0.80000000000000004

In [57]:
#用testing data計算平方誤差
from sklearn.metrics import mean_squared_error

y_predict = regression_model.predict(X_test)
regression_model_mse = mean_squared_error(y_predict, y_test)
regression_model_mse

0.20000000000000001

In [58]:
#將誤差開平方根
import math

math.sqrt(regression_model_mse)

0.4472135954999579

In [59]:
#使用未引入data做測試(取任兩相鄰資料做normalize)
t1 = df.loc[df['日期'] == '2017/10/17'][['成交股數', '成交筆數', '美元／新台幣']]
t2 = df.loc[df['日期'] == '2017/10/16'][['成交股數', '成交筆數', '美元／新台幣']]

#重設資料index
t1.reset_index(drop=True,inplace=True)
t2.reset_index(drop=True,inplace=True)

#格式化資料
t1.loc[0, '成交股數'] = int(t1.loc[0, '成交股數'].replace(',', ''))/int(t2.loc[0, '成交股數'].replace(',', ''))
t1.loc[0, '成交筆數'] = int(t1.loc[0, '成交筆數'].replace(',', ''))/int(t2.loc[0, '成交筆數'].replace(',', ''))
t1.loc[0, '美元／新台幣'] = float(t1.loc[0, '美元／新台幣'])/float(t2.loc[0, '美元／新台幣'])

#預測結果
int(regression_model.predict([t1.loc[0].tolist()]))

0